# Data Setup

In [ ]:
# Some common packages that we may need
from pandas.plotting import scatter_matrix 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Import some common packages
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

def load_police_data():
  crimes=pd.read_csv('./data/Crime_Data.csv')
  return crimes

def load_oridinal_violence_features():
  ordinal_violence=pd.read_csv('./data/crime_categories.csv')
  return ordinal_violence

Now that the data is loaded we want to look at the data and see how it is organized and what groups are being used. We can also get some insight onto if the data may be useful adn we can look and see how we may split or encode our data.

In [ ]:
crimes = load_police_data()
crimes.info()

FileNotFoundError: ignored

In [ ]:
crimes.head(10)

In [ ]:
crimes.info()

# Data Exploration

## Non Numerical Data Exploration

Now lets look a little closer at the non numerical data

In [ ]:
crimes["Offense"].value_counts()

In [ ]:
crimes["StreetName"].value_counts()

In [ ]:
crimes["Agency"].value_counts()

In [ ]:
crimes["DateReported"].value_counts()

In [ ]:
crimes["ReportingOfficer"].value_counts()

There are a lot of categories in the non-numerical data. This is not as worrying as it could be since a lot of the data seems like it is unnecessary for our project. It does not seem like we are going to need Reporting Officer, Agency, and Date Reported. We can just use the hours integer to get the time. 

Small side note: It would be interesting to look up odds of getting arrested on certain streets depending on who is on duty, but that is outside of the scope of our particular project. 

## Numerical Data Exploration

Now we will look at histograms of the numerical data below. From this data we will determine what is important information and what is not. 

In [ ]:
crimes.hist(bins=50, figsize=(20,15))

In [ ]:
crimes['HourReported'].hist(bins=24) # if we use more than 24 bins, the fact that nothing can be reported after 59 minutes makes the bar plot really weird

The histrogram of the Record ID behaved exactly as expected. We assumed that Incident ID would also not be very useful but it is organized by year, which will not really fit into what we are doing for this part of the project, but it would be interesting to look at what crimes have become less frequent over time, and it may actually play a role so even though we initially plan on getting rid of it, we will reevaluate as we continue to work on the project. 
There are some correlations with lower block numbers having more incidents which we did not suspect as a group but more incidents at later hours was something we expected. 

## Null Data

First we will observe the null data and think of a plan to deal with it.

In [ ]:
incomplete_data = crimes[crimes.isnull().any(axis=1)].head()
incomplete_data

In [ ]:
crimes_without_null = crimes.dropna()
crimes_without_null.info()

In [ ]:
crimes.info()

Looking at the data we see that there are about 1300 missing block numbers, I am not sure how cobining this data with the streetnames is going to work but we will try to use an imputer but it may cause some strange behaviors that will need to be looked at later depending on our observations.

In [ ]:
imputer = SimpleImputer(strategy="median")

Here we are going to look at imputing the data but we decided we would get rid of the values that do not matter first. 

In [ ]:
crimes.info()

In [ ]:
crimes_lean = crimes.drop(["ReportingOfficer","IncidentID","Agency","RecordID",],axis=1)

In [ ]:
crime_num = crimes_lean.drop(["Offense","DateReported","StreetName"], axis=1)

In [ ]:
imputer.fit(crime_num)

Just want to look at the data in the imputed state

In [ ]:
imputer.statistics_

In [ ]:
crime_num.median().values

Now we need to deal with the categorical data. This is where some big adjustments to what we have input need to happen. 

# Data Cleaning

First, we are going to drop the features we determined are not particularly useful

In [ ]:
crimes_lean = crimes.drop(["ReportingOfficer","IncidentID","Agency","RecordID",],axis=1)

## Offense Encoding

In [ ]:
crime_offense_cat = crimes[["Offense"]]
crime_offense_cat.head(10)

We are going to have to label these by hand by the look of it. There's no way to infer how dangerous an offense is by machine, unless perhaps by some sophisticated NLP algorithm which would take longer to write and design than it would to do this project most likely.

It is important to acknowledge the fact that our labelling of the data is an inherent source of bias. Unfortunately, if we wish to use this data for predicting safety, it will be necessary as a trade-off, because it would just be too innacurrate to base a decision on walking safety on occurence of trafic law violations or something of that nature. Thus, this is a tradeoff that has to be acknowledged as an inherent source of bias since this is not really a clear-cut line between "violent" and "non-violent". Additionally, there seem to be some categories that we may or may not actually be able to interpret clearly, such as `'Tactical Crime Initiative - TCI'`, which may mean something to the police department, but has litte meaning for us, unless it's something we can find on Google. There may be very few features with these categorizations however, so this may be a case where we can simply drop those records.

. . .  time elapses . . .




Having now hand-jammed an ordinal encoding for the categorical offense data, we can load it and see if we can't use this engineered feature to our advantage.

In [ ]:
ord_offenses = load_oridinal_violence_features()
ord_offenses.head()

Unfortunately, this kind of solution is very prone to error, and introduces much inherent bias. This is probably something that it would be good to ask the police department to do, since they actually understand what all of the offences are and how dangerous they really are. Sadly, we are not the police force and they probably would not have time for such a thing if we were to ask them to do it.

The next task is to try and get this data mapped into the main data set. Notably, `ord_offenses` only has one entry for each kind of offense, so we will have to use some kind of pandas trick here. Or maybe just a good old fashioned nested for loop . . . Parallelization! I blow my nose at you!

In [ ]:
# first, we'll add a duplicate of the `Offense` column for the `danger` feature
crimes_lean['Danger'] = crimes_lean.loc[:, 'Offense']

# next, we'll turn the `ord_offenses` data frame into a dictionary
danger_dict = dict(zip(ord_offenses.Offense, ord_offenses.Danger_Level))
# print(danger_dict)

# finally, we can map the new `Danger` column to it's categorical value
crimes_lean['Danger'] = crimes_lean['Danger'].map(danger_dict).map(lambda x: int(x >= 3))

# now, we can take a peek at our lovely new column!
crimes_lean.head()


In [ ]:
crimes_lean['Danger'].hist(bins=2)

## Date Conversion

Another problem we need to deal with is converting the date to some sort of usable numerical format, such as milliseconds since the last unix epoch. Let's examine this feature again.

In [ ]:
crimes_lean['DateReported'].head()

 Commence string munging tricks . . . or maybe we'll just find a library like [`dateutil`](https://labix.org/python-dateutil#head-a23e8ae0a661d77b89dfb3476f85b26f0b30349c)

Yeah, the library is totally the way to go. Of course.

Now, just a matter of using a map or something like that to convert the thing!

In [ ]:
import dateutil as du
import datetime as dt

# let's start by making a copy of the column like before
crimes_lean['Epoch'] = crimes_lean.loc[:, 'DateReported']

# next, we need a function that can be mapped over the column to do the conversion.
def date_converter(in_str):
    date = du.parser.parse(in_str)
    return date

# finally, we can map the function across the duplicate column to get the unix epoch
# crimes_lean['Epoch'] = crimes_lean['Epoch'].map(date_converter)
crimes_lean['Weekday'] = crimes_lean['Epoch'].map(lambda x: date_converter(x).weekday())
crimes_lean['Year'] = crimes_lean['Epoch'].map(lambda x: date_converter(x).year)
crimes_lean['Month'] = crimes_lean['Epoch'].map(lambda x: date_converter(x).month)
crimes_lean['Day'] = crimes_lean['Epoch'].map(lambda x: date_converter(x).day)

# now, we can take a peek at our lovely new column!
crimes_lean.head()
   

Booyah!

Now we can actually look at these times numerically to see trends with time for example!

## Location Encoding

### Road Centerlines Dataset

The first step to this process is going to be to find some sort of way to translate the block system used by the Charlottesville Police Department to something that is practically useful. To this end, we have downloaded and will utilize the [Road Centerlines](https://opendata.charlottesville.org/datasets/charlottesville::road-centerlines-block-level/about) dataset.

In [ ]:
!pip install geopandas
import geopandas as gpd                         # this allows us to use standard geospatial formats (such as the geojson that we downloaded)

road_lines_full = gpd.read_file("./data/Road_Centerlines_(Block_Level).geojson")             # Note that `road_lines` is a geodataframe, and is not json-esque
print(road_lines_full.head())

Oh, dear. I thought these were supposed to be centerlines . . . It looks like these are linestrings that describe the left and right edges of the road. \*sigh\*

I suppose we could just pretend one side of the road doesn't exist . . .

It could also be that those with a L / R  attribute are for split highways with a median in between. Regardless, we don't have that level of detail in our crimes data if I am not mistaken.

One thing we can try to simplify this scheme is to use the observation that the finest level of detail we have in the crimes data set is by the block number and street name. Therefore, we are not losing any precision if we were to take the center point of each block, and use that as the coordinate for any crime on that block. To do this, we can take the linestrings formed by the left and right halves of the data, draw additional lines between their starts and ends, which will turn each block into a polygon. We can then find the centerpoint of the polygon to determine a single lat-lon point for the block.

However, because time is short and `geopandas` is complex, we are instead just going to keep only the left side of each block, and we will use the center point of this line string as the geometric coordinate.

In [ ]:
# we would like to make sure that there isn't anything besides left and right
print(road_lines_full.info())
print(road_lines_full.value_counts(["Side"]))

Notably, there are more left-hand sides than right hand sides. I am guessing/assuming this is because one-way streets only have a left-hand side. This doesn't really make sense though, because we really only drive on the right-hand side. Regardless, dropping all of the right-hand rows will work for now.

In [ ]:
# we start by dropping all of the right hand rows.

road_lines_naive = road_lines_full[road_lines_full.Side != "R"]

The center point can then be found with the this [function](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.centroid.html)

In [ ]:
# we can start by making a copy of the geometry column
road_lines_naive['point'] = road_lines_naive.loc[:, 'geometry']

# then we can make our mapping function
def naive_center_finder(linestring):
    return gpd.GeoSeries([linestring]).centroid[0]

# and then we can map our handy-dandy function over this column
road_lines_naive['point'] = road_lines_naive['point'].map(naive_center_finder)

# and finally see if it worked
print(road_lines_naive.head(10))

weeeellll . . . I suppose it worked. We have no real way of knowing whether those are actually the centerpoints I would hope to see, but hey, at least it's something.

### Using the Road Center Points in the Crimes Data

To actually use what we just made, we can now use this data set to add a coordinate to each crime!

In [ ]:
crimes_lean = crimes_lean.dropna()

In [ ]:
# We are going to start by concatenating the block number and street name into one field.
# hopefully these will be identical for both data sets.
crimes_lean['road_location'] = crimes_lean['StreetName'] + crimes_lean['BlockNumber'].astype(int).astype(str)
# print(crimes_lean.head())

road_lines_naive['road_location'] = road_lines_naive['Street'] + road_lines_naive['Block'].astype(int).astype(str)
print(road_lines_naive.head())

In [ ]:
# next we can make a dictionary out of the road_lines_naive data set
road_centers_dict = dict(zip(road_lines_naive.road_location, road_lines_naive.point))

# and then make a point column for the crimes data
crimes_lean['Point'] = crimes_lean.loc[:, 'road_location']

# and finally we can use the dictionary to map over this column to get our points
crimes_lean['Point'] = crimes_lean['Point'].map(road_centers_dict)

print(crimes_lean.info())

We have successfully given each crime a lat-lon coordinate, which is super fantastic! Even if we had to lose about 6k crimes in the process, and even if the point isn't as accurate as it could be since we only used half of the street.

Also, we really need that point to be split into latitude and longitude columns practically speaking . . .

In [ ]:
# We can do that very easily using geopandas as well
crimes_lean = crimes_lean.dropna()

crimes_lean['Longitude'] = crimes_lean['Point'].apply(lambda point: point.x)
crimes_lean['Latitude'] = crimes_lean['Point'].apply(lambda point: point.y)

print(crimes_lean.head())

# Modeling

In [ ]:
# and now, lets grab just the really useful features we have constructed and see if we get some interesting regression results out of it
crimes_sparse = crimes_lean[['Weekday', 'Year', 'Month', 'Day', 'Latitude', 'Longitude', 'Danger']]

print(crimes_sparse.head())

Wow, we say "sparse", but we should probably say "Charlie-Brown-Christmas-Tree-Esque".

![charlie-brown-tree](https://wallpapercave.com/wp/9Dvub0b.jpg)

It's funny how after extracting the essential information out of the data, we really have very, very little to work with for each record. Just when the reported happened, where the reported incident happend, and how bad the incident was on a scale of 0 to 4. Hopefully that's enough to do some reasonable machine learning . . .

### Visualization Detour

Let's do a quick 3D graph with a random subset of our data to try and get a feel for whether we can legitimately try to use a linear regression to solve our problem.

In [ ]:
sample_crimes = crimes_sparse.sample(frac=0.1, random_state=42)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(sample_crimes['Longitude'], sample_crimes['Latitude'], sample_crimes['Day'], c=sample_crimes['Danger'], cmap='jet')

NameError: ignored

In the above plot, the axes are longitude, latitude, and time for x, y, and z respectively, and the color of each dot is based on its danger level from 0 to 4. Although this isn't the greatest plot, I tend to think this data is _not_ going to be linearly separable by any stretch of the imagination.

So the question then becomes, if we can't do a linear regression, what should we do instead? We have just graphed the full dimensionality of our space, so SVM may not be sufficient here as there doesn't seem to be any sort of clear pattern whatsoever. SVM might still be worth looking into however, especially since it can be used to map the feature space to higher dimensions.

An interesting thing to consider here is that the feature we would want to predict, `Danger`, can be treated as either a categorical or a numeric value. What this means practically is that we can treat this problem not so much as a regression, but rather as a classification problem. Unfortunately, K-Means and other clustering methods will also probably fail to be of use here since the dimensionality is so very low, although they may still be worth trying. 

This leaves us with Neural Nets. A deep neural net may be able to make sense of this low dimensionality feature space in a meaningful way.

### Neural Net mk. I

Let's give the neural net a go.

We can make a validation set and a training set.

In [ ]:
# and we can also standard scale our features
from sklearn import pipeline
from sklearn.compose import ColumnTransformer
# pipeline to standard scale the input
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_features = ['Weekday', 'Year', 'Month', 'Day']
numeric_features = ['Latitude', 'Longitude']
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

# # full pipeline:
# full_pipeline = ColumnTransformer([
#         ("num", num_pipeline, num_attribs),
#     ])
nn_1_pipeline = preprocessor
# to_be_X = crimes_sparse.drop("Danger", axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X = crimes_sparse.drop('Danger', axis=1)
Y = crimes_sparse['Danger']

nn_1_train_full_X, nn_1_test_X, nn_1_train_full_Y, nn_1_test_Y = train_test_split(X, Y, test_size=0.2, random_state=42)
nn_1_train_full_X = nn_1_pipeline.fit_transform(nn_1_train_full_X, nn_1_train_full_Y)
nn_1_test_X = nn_1_pipeline.fit(nn_1_test_X, nn_1_test_Y)
nn_1_train_x, nn_1_val_x, nn_1_train_y, nn_1_val_y = train_test_split(nn_1_train_full_X, nn_1_train_full_Y, test_size=0.1, random_state=42)

print(nn_1_train_x.shape)

print(nn_1_val_x.shape)

(12448, 58)
(1384, 58)


In [ ]:
import tensorflow as tf
import keras

nn_1 = tf.keras.models.Sequential(layers=[
    keras.layers.Flatten(),        # input layer
    keras.layers.Dense(units=100, activation='relu'),  # hidden layer
    # keras.layers.Dense(units=1750, activation='relu'), # hidden layer
    keras.layers.Dense(units=100, activation='relu'), # hidden layer
    # keras.layers.Dropout(0.5),
    keras.layers.Dense(units=2, activation='softmax') # output softmax layer
])

In [ ]:
nn_1_epochs = 50 
# nn_1_opt = tf.keras.optimizers.SGD(learning_rate=0.2, momentum=0.1)
nn_1_opt = tf.keras.optimizers.Nadam()
nn_1_loss = ["sparse_categorical_crossentropy"]
nn_1_mtrx = ["accuracy"] 
nn_1.compile(loss= nn_1_loss, optimizer = nn_1_opt, metrics = nn_1_mtrx)
history = nn_1.fit(nn_1_train_x.toarray(), nn_1_train_y, validation_data=(nn_1_val_x.toarray(), nn_1_val_y), epochs = nn_1_epochs)

Epoch 1/50
389/389 [==============================] - 3s 3ms/step - loss: 0.6495 - accuracy: 0.6445 - val_loss: 0.6448 - val_accuracy: 0.6575
Epoch 2/50
389/389 [==============================] - 1s 3ms/step - loss: 0.6403 - accuracy: 0.6466 - val_loss: 0.6428 - val_accuracy: 0.6582
Epoch 3/50
389/389 [==============================] - 1s 3ms/step - loss: 0.6335 - accuracy: 0.6504 - val_loss: 0.6452 - val_accuracy: 0.6568
Epoch 4/50
389/389 [==============================] - 1s 3ms/step - loss: 0.6256 - accuracy: 0.6554 - val_loss: 0.6436 - val_accuracy: 0.6539
Epoch 5/50
389/389 [==============================] - 1s 3ms/step - loss: 0.6168 - accuracy: 0.6663 - val_loss: 0.6499 - val_accuracy: 0.6517
Epoch 6/50
389/389 [==============================] - 1s 3ms/step - loss: 0.6066 - accuracy: 0.6735 - val_loss: 0.6551 - val_accuracy: 0.6329
Epoch 7/50
389/389 [==============================] - 1s 3ms/step - loss: 0.5969 - accuracy: 0.6838 - val_loss: 0.6590 - val_accuracy: 0.6236
Epoch 

KeyboardInterrupt: ignored

In [ ]:
nn_1.summary()

### SVM Model

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from scipy.stats import reciprocal, uniform, randint


X_train = nn_1_train_full_X.toarray()
y_train = nn_1_train_full_Y
X_test  = nn_1_val_x.toarray()
y_test  = nn_1_val_y

In [ ]:
# linear
svm_linear = SVC(kernel="linear", C=1.0)
svm_linear.fit(X_train, y_train)

# print confusion matrix, precision score, and recall score
y_pred = svm_linear.predict(X_test)
confusion = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print(f'Confusion Matrix: {confusion}')
print(f'Precision Score: {precision}')
print(f'Recall Score: {recall}')

Confusion Matrix: [[  0 474]
 [  0 910]]
Precision Score: 0.6575144508670521
Recall Score: 1.0


In [ ]:
# Polynomial
svm_poly = SVC(kernel="poly", C=1.0)
svm_poly.fit(X_train, y_train)

SVC(kernel='poly')

In [ ]:
# Gaussian RBF 
svm_rbf = SVC(kernel="rbf", C=1.0)
svm_rbf.fit(X_train, y_train)

SVC()

In [ ]:
classifiers = [svm_linear, svm_poly, svm_rbf]
names = ['linear', 'poly', 'rbf']

param_distributions = [{"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 100)},
                       {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 100), "degree": randint(2, 10)},
                       {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 100)}]

for i in [0, 1, 2]:
  rnd_search_cv = RandomizedSearchCV(classifiers[i], param_distributions[i], n_iter=5, verbose=3, cv=3)
  rnd_search_cv.fit(X_train, y_train)
  print('\n\n-----------------------------------------------------------------')
  print(f'svc with {names[i]} kernel:')
  print(f'best score: {rnd_search_cv.best_score_}')
  print(f'best_estimator: {rnd_search_cv.best_estimator_}')

  # print confusion matrix, precision score, and recall score
  y_pred = rnd_search_cv.predict(X_test)
  confusion = confusion_matrix(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  print(f'Confusion Matrix: {confusion}')
  print(f'Precision Score: {precision}')
  print(f'Recall Score: {recall}')

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV 1/3] END C=19.34347898661638, gamma=0.03625617634576227;, score=0.649 total time=  25.1s
[CV 2/3] END C=19.34347898661638, gamma=0.03625617634576227;, score=0.649 total time=  23.5s
[CV 3/3] END C=19.34347898661638, gamma=0.03625617634576227;, score=0.649 total time= 1.3min
